In [1]:
import numpy as np
import astropy.io.fits as pyfits
from astropy.table import Table
import copy
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import sys, os
import subprocess

# Multiband

In [2]:
# change column names
translate = {'x': 'x_fit',
             'x_err': 'x_fit_err',
             'y': 'y_fit',
             'y_err': 'y_fit_err',
             'flux': 'flux_tot',
             'flux_err': 'flux_tot_err',
             'col1': 'ID',
             'col2': 'X',
             'col3': 'Y'
            }

In [3]:
# columns to drop
#drop_cols = ['x','y','x_err','y_err']
drop_cols = ['detection_id', 'group_id', 'ra', 'ra_err', 'dec', 'dec_err', 'X_rad', 'X_rad_err', 
             'X_sersic', 'X_sersic_err', 'angle', 'angle_err', 'X_bt', 'X_bt_err', 'X_disk_radius', 
             'X_disk_radius_err', 'rel_s', 'rel_s_err', 'RA', 'RA_err', 'Dec', 'Dec_err', 'X_disk_effR', 
            'X_disk_effR_err', 'X_st', 'X_st_err', 'auto_flux', 'auto_flux_err', 'auto_mag', 
             'auto_mag_err', 'auto_flags', 'flux_radius', 'snrratio', 'Separation']

bands = ['vis', 'u_lsst', 'g_lsst', 'r_lsst', 'i_lsst', 'z_lsst', 'y_nir', 'j_nir', 'h_nir']
for b in bands:
    drop_cols += ['aper_{}_flux'.format(b), 'aper_{}_flux_err'.format(b), 'aper_{}_mag'.format(b), 
              'aper_{}_mag_err'.format(b), 'X_bt_{}'.format(b), 'X_bt_{}_err'.format(b), 
                  'aper_{}_flags'.format(b)]

In [4]:
for col in drop_cols:
    print(col)

detection_id
group_id
ra
ra_err
dec
dec_err
X_rad
X_rad_err
X_sersic
X_sersic_err
angle
angle_err
X_bt
X_bt_err
X_disk_radius
X_disk_radius_err
rel_s
rel_s_err
RA
RA_err
Dec
Dec_err
X_disk_effR
X_disk_effR_err
X_st
X_st_err
auto_flux
auto_flux_err
auto_mag
auto_mag_err
auto_flags
flux_radius
snrratio
Separation
aper_vis_flux
aper_vis_flux_err
aper_vis_mag
aper_vis_mag_err
X_bt_vis
X_bt_vis_err
aper_vis_flags
aper_u_lsst_flux
aper_u_lsst_flux_err
aper_u_lsst_mag
aper_u_lsst_mag_err
X_bt_u_lsst
X_bt_u_lsst_err
aper_u_lsst_flags
aper_g_lsst_flux
aper_g_lsst_flux_err
aper_g_lsst_mag
aper_g_lsst_mag_err
X_bt_g_lsst
X_bt_g_lsst_err
aper_g_lsst_flags
aper_r_lsst_flux
aper_r_lsst_flux_err
aper_r_lsst_mag
aper_r_lsst_mag_err
X_bt_r_lsst
X_bt_r_lsst_err
aper_r_lsst_flags
aper_i_lsst_flux
aper_i_lsst_flux_err
aper_i_lsst_mag
aper_i_lsst_mag_err
X_bt_i_lsst
X_bt_i_lsst_err
aper_i_lsst_flags
aper_z_lsst_flux
aper_z_lsst_flux_err
aper_z_lsst_mag
aper_z_lsst_mag_err
X_bt_z_lsst
X_bt_z_lsst_err
aper_z

In [5]:
rdir = "results/multiband_noiter_noresamp_SEv0.16.cat"
list_dir = "truth/input_positions_allfields"
out_dir = "entry/Multiband"

rfile = os.path.join(rdir,"full_multiband_B+D_good_detection.cat.fits")
list_file = os.path.join(list_dir,"input_positions_vis_0_all.list")
out_file = os.path.join(out_dir,"dsersic_multiband_0.fits")

# match
cmd = 'stilts tmatch2 in1={} ifmt1=ascii in2={} out={} matcher=2d values1="col2 col3" values2="x y" \
params=3 join=all1'.format(list_file, rfile, out_file)
subprocess.call(cmd, shell=True)

t = Table.read(out_file)

# check that we don't have double matches
if len(t[t['source_id']>-1]) > len(np.unique(t['source_id'])-2):
       print("Warning: Looks like you have some double matches")
        
# Need to change angle from rad to deg.?
if 'angle_deg' not in t.colnames:
    t['angle_deg'] = t['angle']*180/np.pi
    t['angle_deg_err'] = t['angle_err']*180/np.pi
    t['angle_deg'][t['angle_deg'] < -90] += 180
    t['angle_deg'][t['angle_deg'] > 90] -= 180

# delete the columns we don't need
for col in drop_cols:
    try:
        del t[col]
    except:
        pass

# rename columns
for item in translate.items():
    try:
        t[item[0]].name = translate[item[0]]
    except:
        pass

# add in flag values for unmatched objects
for col in t.colnames:
    t[col][np.isnan(t[col])] = -9999
    t[col][t[col].mask==True] = -9999            

# save file
t.write(out_file, overwrite=True)

